Anmerkung: Dieses Notebook ist Teil von https://github.com/SamuelVilz/anki-german-sign-language und es ergibt wenig Sinn, es unabhängig von den dortigen Dateien zu verwenden.

# Imports

In [1]:
import sys

In [2]:
# ! pip install genanki

In [3]:
import genanki

In [4]:
import random

In [5]:
import re

# Card-HTMLs hardcoden

In [6]:
def getVideoHTMLStringFront(videonumber):
    returnstring = '<div class="container">\n'
    returnstring += '<iframe width="400" height="300" frameborder="0" src="{{Video '+str(videonumber)+'}}" allow="autoplay; encrypted-media" allowfullscreen></iframe>\n'
    returnstring += '{{#Video '+str(videonumber)+' contains text}}\n'
    returnstring += '<div class="cover"></div>\n'
    returnstring += '{{/Video '+str(videonumber)+' contains text}}\n'
    returnstring += '</div>'
    return returnstring

In [7]:
meaningFront = '{{Meaning}}\n\n{{#Note}}\n<div class="note">{{Note}}</div>\n{{/Note}}'

In [8]:
videoBackside = '{{FrontSide}}\n\n<hr id=answer>\n\n{{Meaning}}\n\n{{#Note}}\n<div class="note">{{Note}}</div>\n{{/Note}}'

In [9]:
def getMeaningHTMLStringBack():
    returnstring = ''
    for i in range(1,7):
        returnstring+='\n{{#Video '+str(i)+'}}\n<iframe width="400" height="300" frameborder="0" src="{{Video '+str(i)+'}}" allow="autoplay; encrypted-media" scrolling="no"></iframe>\n{{/Video '+str(i)+'}}\n'
    return returnstring
meaningBack = '{{FrontSide}}\n\n<hr id=answer>\n' + getMeaningHTMLStringBack()

In [10]:
stylingstring = ''
with open('Styling.txt', 'r') as f:
    stylingstring=f.read()

In [11]:
templatesArray = []
for i in range(1,7):
    dicti = {}
    dicti['name'] = 'Video '+str(i)
    dicti['qfmt'] = getVideoHTMLStringFront(i)
    dicti['afmt'] = videoBackside
    templatesArray.append(dicti)
dictmeaning = {}
dictmeaning['name'] = 'Meaning'
dictmeaning['qfmt'] = meaningFront
dictmeaning['afmt'] = meaningBack
templatesArray.append(dictmeaning)

# Erstellung des Models

In [12]:
dgs_model = genanki.Model(
  model_id=5,
  name='DGS Model',
  fields=[
    {'name': 'Meaning'},
    {'name': 'Note'},
    {'name': 'Video 1'},
    {'name': 'Video 2'},
    {'name': 'Video 3'},
    {'name': 'Video 4'},
    {'name': 'Video 5'},
    {'name': 'Video 6'},
    {'name': 'Video 1 contains text'},
    {'name': 'Video 2 contains text'},
    {'name': 'Video 3 contains text'},
    {'name': 'Video 4 contains text'},
    {'name': 'Video 5 contains text'},
    {'name': 'Video 6 contains text'}
  ],
  templates=templatesArray,
  css = stylingstring
)

# Ein Beispieldeck erzeugen

In [13]:
# Diese Methode ist später vor allem deshalb nützlich, weil man beim Eingeben der Werte die Parameternamen sieht, statt
# sie blind in eine Liste zu packen
def noteFields(meaning,
           video1,
           note = '',
           video2 = '',
           video3 = '',
           video4 = '',
           video5 = '',
           video6 = '',
           video1t = '',
           video2t = '',
           video3t = '',
           video4t = '',
           video5t = '',
           video6t = ''):
    return [meaning,
            note,
            video1,
            video2,
            video3,
            video4,
            video5,
            video6,
            video1t,
            video2t,
            video3t,
            video4t,
            video5t,
            video6t]

In [14]:
dankenote = genanki.Note(
  model=dgs_model,
  fields=noteFields(meaning = 'danke',
                    video1 = 'https://signdict.org/embed/1360-danke/video/151',
                    video2 = 'https://signdict.org/embed/1360-danke/video/1526'))

In [15]:
merkelnote = genanki.Note(
  model=dgs_model,
  fields=noteFields(meaning = 'Merkel',
                    video1 = 'https://signdict.org/embed/2456-merkel/video/2757'))

In [16]:
dnkmrkl_deck = genanki.Deck(
  6,
  'Danke Merkel')

dnkmrkl_deck.add_note(dankenote)
dnkmrkl_deck.add_note(merkelnote)

In [17]:
genanki.Package(dnkmrkl_deck).write_to_file('Custom Decks/1 dnkmrkl.apkg')

# Einträge parsen

**Achtung!** Hierfür muss die Datei "entries" im gleichen Ordner wie das Notebook vorliegen. Zur Erzeugung (und Aktualisierung) dieser Datei muss der Ruby-Code `crawl.rb` ausgeführt werden.

In [18]:
f = open("entries", "r", encoding="utf-8")
entrylist = []
for x in f:
  entrylist.append(x[:-1].replace("/entry/", "/embed/").split('\t'))
f.close()

In [19]:
[x for x in entrylist if x[1]=='Morgen']

[['/embed/507-morgen/video/573', 'Morgen', 'kommender Tag'],
 ['/embed/507-morgen/video/574', 'Morgen', 'kommender Tag'],
 ['/embed/507-morgen/video/575', 'Morgen', 'kommender Tag'],
 ['/embed/507-morgen/video/576', 'Morgen', 'kommender Tag'],
 ['/embed/507-morgen/video/2824', 'Morgen', 'kommender Tag'],
 ['/embed/506-morgen/video/572', 'Morgen', 'Tageszeit'],
 ['/embed/506-morgen/video/2825', 'Morgen', 'Tageszeit']]

In [20]:
f = open("containtext", "r", encoding="utf-8")
textlist = []
for x in f:
  textlist.append(x[:-1].split()[0])
f.close()
textlist[:5]

['21', '36', '40', '114', '132']

In [21]:
entry_note_keys = list(sorted(set(map(lambda x: tuple(x[1:]), entrylist))))
entry_note_keys[:5]

[('0',), ('1',), ('10',), ('100',), ('1000',)]

In [22]:
my_deck = genanki.Deck(
  7,
  'SignDict Alles')
allnotelist = [] # for debugging
for key in entry_note_keys:
    key_data = [x for x in entrylist if tuple(x[1:]) == key]
    videos = []
    for i in range(6):
        videos.append('https://signdict.org'+key_data[i][0] if len(key_data)>i else '')
    fields = noteFields(meaning = key_data[0][1],
                        note = key_data[0][2] if len(key_data[0])>2 else '',
                        video1 = videos[0],
                        video2 = videos[1],
                        video3 = videos[2],
                        video4 = videos[3],
                        video5 = videos[4],
                        video6 = videos[5],
                        video1t = "yes" if videos[0].split('/')[-1] in textlist else "",
                        video2t = "yes" if videos[1].split('/')[-1] in textlist else "",
                        video3t = "yes" if videos[2].split('/')[-1] in textlist else "",
                        video4t = "yes" if videos[3].split('/')[-1] in textlist else "",
                        video5t = "yes" if videos[4].split('/')[-1] in textlist else "",
                        video6t = "yes" if videos[5].split('/')[-1] in textlist else "")
    thisnote = genanki.Note(
      model=dgs_model,
      fields=fields)
    allnotelist.append(fields) # for debugging
    my_deck.add_note(thisnote)

In [23]:
allnotelist[:3]

[['0',
  '',
  'https://signdict.org/embed/2636-0/video/2962',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['1',
  '',
  'https://signdict.org/embed/181-1/video/206',
  'https://signdict.org/embed/181-1/video/1668',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['10',
  '',
  'https://signdict.org/embed/826-10/video/940',
  'https://signdict.org/embed/826-10/video/4221',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '']]

In [24]:
genanki.Package(my_deck).write_to_file('Custom Decks/2 SignDictAlles.apkg')

# Custom Decks einfacher erstellen

Zunächst ein Wort der Vorsicht: Wenn ein Deck in Anki importiert wird, das Karten enthält, die bereits in einem anderen Deck vorliegen, dann werden diese Karten nicht im neuen Deck auftauchen.

## Noch ein Blick auf den Umgang mit den Daten

Wie soll der Nutzer seine gewünschten Karten angeben? Wie behandeln wir Fälle, in denen "Meaning" nicht eindeutig ist?

In [25]:
someset = list(sorted(set([tuple(x[1:]) for x in entrylist
                           if len([y for y in entrylist if x[1]==y[1] and (len(x)!=len(y) or (len(x)>2 and x[2]!=y[2]))])
                               >0])))

In [26]:
someset[:10]

[('Amerika',),
 ('Amerika', 'Kontinent'),
 ('Bahn',),
 ('Bahn', 'Parcours'),
 ('Bank', 'Geld'),
 ('Bank', 'Park'),
 ('Bar',),
 ('Bar', 'Geld'),
 ('Bedienung',),
 ('Bedienung', 'Handhabung')]

In [27]:
[x for x in someset if 'Morgen' in x[0]]

[('Morgen', 'Tageszeit'), ('Morgen', 'kommender Tag')]

## Funktion zur Erstellung

In [28]:
def createDeck(deckname, filename, keys):
    custom_deck = genanki.Deck(
      random.randrange(1 << 30, 1 << 31),
      deckname)
#     allnotelist = [] # for debugging
    for key in keys:
        key_data = [x for x in entrylist if tuple(x[1:]) == key]
        videos = []
        for i in range(6):
            videos.append('https://signdict.org'+key_data[i][0] if len(key_data)>i else '')
        fields = noteFields(meaning = key_data[0][1],
                            note = key_data[0][2] if len(key_data[0])>2 else '',
                            video1 = videos[0],
                            video2 = videos[1],
                            video3 = videos[2],
                            video4 = videos[3],
                            video5 = videos[4],
                            video6 = videos[5],
                            video1t = "yes" if videos[0].split('/')[-1] in textlist else "",
                            video2t = "yes" if videos[1].split('/')[-1] in textlist else "",
                            video3t = "yes" if videos[2].split('/')[-1] in textlist else "",
                            video4t = "yes" if videos[3].split('/')[-1] in textlist else "",
                            video5t = "yes" if videos[4].split('/')[-1] in textlist else "",
                            video6t = "yes" if videos[5].split('/')[-1] in textlist else "")
        thisnote = genanki.Note(
          model=dgs_model,
          fields=fields)
#         allnotelist.append(fields) # for debugging
        custom_deck.add_note(thisnote)
    genanki.Package(custom_deck).write_to_file('Custom Decks/'+filename+'.apkg')

## Beispiel: Alphabet-Deck

In [29]:
singlecharacterentrys = [x[1] for x in entrylist if len(x[1])==1]
singlecharacterentrys[:5], "...", singlecharacterentrys[-5:]

(['a', 'b', 'c', 'd', 'e'], '...', ['7', '7', '8', '9', '9'])

Diese Liste enthält leider auch Ziffern, daher müssen wir uns regulärer Ausdrücke (RegEx) bedienen.

In [30]:
letterentries = [x[1] for x in entrylist if re.fullmatch('[A-Za-zßÄÖÜäöü]',x[1])]
len(letterentries), letterentries[:3],'...',letterentries[-3:]

(26, ['a', 'b', 'c'], '...', ['x', 'y', 'z'])

Umlaute und ß gibt es bisher nicht, aber dafür kann unser Code ja nichts. "sch" hätten wir gerne trotzdem noch als Teil des Alphabets:

In [31]:
letterentries = [x[1] for x in entrylist if re.fullmatch('[A-Za-zßÄÖÜäöü]|sch',x[1])]
len(letterentries), letterentries[:3],'...',letterentries[-3:]

(27, ['a', 'b', 'c'], '...', ['x', 'y', 'z'])

Jetzt generieren wir die notwendigen Keys und überreichen diese an die oben definierte Funktion

In [32]:
letterkeys = list(sorted(set(map(lambda x: tuple(x[1:]),
                                [x for x in entrylist if re.fullmatch('[A-Za-zßÄÖÜäöü]|sch',x[1])]))))
letterkeys[:5]

[('a',), ('b',), ('c',), ('d',), ('e',)]

In [33]:
createDeck('Fingeralphabet', '3 Fingeralphabet', letterkeys)

## Beispiel: Zahlen von 1 bis 10

In [34]:
numberentries = [x[1] for x in entrylist if re.fullmatch('[1-9]|10',x[1])]
len(numberentries), numberentries[:3],'...',numberentries[-3:]

(16, ['1', '1', '10'], '...', ['8', '9', '9'])

Das sind einige Duplikate. Mal Übersicht schaffen:

In [35]:
list(sorted(set(numberentries)))

['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

Gut, das wollen wir

In [36]:
numberkeys = list(sorted(set(map(lambda x: tuple(x[1:]),
                                [x for x in entrylist if re.fullmatch('[1-9]|10',x[1])]))))
numberkeys

[('1',),
 ('10',),
 ('2',),
 ('3',),
 ('4',),
 ('5',),
 ('6',),
 ('7',),
 ('8',),
 ('9',)]

In [37]:
createDeck('Zahlen bis 10', '4 Zahlen bis 10', numberkeys)

## Beispiel: Alle Zahlen

In [38]:
allnumberentries = [x[1] for x in entrylist if re.fullmatch('[0-9]*',x[1])]
len(allnumberentries), allnumberentries[:3],'...',allnumberentries[-3:]

(82, ['0', '1', '1'], '...', ['9', '90', '90'])

In [39]:
allnumberkeys = list(sorted(set(map(lambda x: tuple(x[1:]),
                                [x for x in entrylist if re.fullmatch('[0-9]*',x[1])]))))
allnumberkeys

[('0',),
 ('1',),
 ('10',),
 ('100',),
 ('1000',),
 ('11',),
 ('12',),
 ('13',),
 ('14',),
 ('15',),
 ('16',),
 ('17',),
 ('18',),
 ('19',),
 ('2',),
 ('20',),
 ('200',),
 ('2000',),
 ('21',),
 ('22',),
 ('23',),
 ('24',),
 ('25',),
 ('26',),
 ('27',),
 ('28',),
 ('29',),
 ('3',),
 ('30',),
 ('31',),
 ('32',),
 ('33',),
 ('34',),
 ('35',),
 ('36',),
 ('37',),
 ('38',),
 ('39',),
 ('4',),
 ('40',),
 ('5',),
 ('50',),
 ('6',),
 ('60',),
 ('6000',),
 ('66',),
 ('7',),
 ('70',),
 ('8',),
 ('80',),
 ('9',),
 ('90',)]

In [40]:
createDeck('Alle Zahlen', '5 Alle Zahlen', allnumberkeys)

## Beispiel: Selbstgewählte Einträge

### Welche Einträge wollen wir?

Hierfür erstellen wir zunächst eine handliche Liste mit allen Vokabeln, die wir in einem Deck haben wollen:

In [41]:
vocablist = [
    'ich',
    'du',
    'er',
    'sie',
    'es',
    'wir',
    'ihr',
    'gehörlos',
    'wir beide',
    'mein',
    'dein',
    'sein, ihr',
    'unser',
    'euer',
    'Name',
    'was',
    'Körper',
    'gut',
    "wie geht's",
    'nicht gut gehen',
    'schlecht',
    'super',
    'einigermaßen',
    'hallo',
    'tschüss',
    'taub',
    'hörend',
    'ertaubt',
    'Stimme weglassen',
    'vorstellen',
    'schwerhörig',
    'kennenlernen',
    'Fingeralphabet',
    'Vorname',
    'Nachname',
    'Gebärdensprache',
    'Gebärde',
    'Sprache',
    'nach',
    'wer',
    'wo',
    'Wort',
    'Vokabeln',
    'Satz',
    'Grammatik',
    'Mimik',
    'müde',
    'prima',
    'übel',
    'Kopfschmerzen',
    'Schwindel',
    'mittel',
    'wohl',
    'fit'
]

Dann lesen wir die Liste ein und lassen uns ausgeben, wenn es Probleme oder Mehrdeutigkeiten gibt:

In [42]:
def getResults(vocab):
    print("_____________")
    perfectmatches = list(sorted(set([str(x[1:]) for x in entrylist if vocab.lower() == x[1].lower()])))
    if len(perfectmatches)>0:
        print("'" + vocab + "' -> " + str(perfectmatches))
        return
    okaymatches = list(sorted(set([str(x[1:]) for x in entrylist if vocab.lower() in x[1].lower()])))
    if len(okaymatches)>0:
        if len(okaymatches)>10:
            okaymatches = okaymatches[:10] + ['...']
        print("'" + vocab + "' wurde gefunden in: " + str(okaymatches))
        return
    print("'" + vocab + "' wurde nicht gefunden")
    
for v in vocablist:
    getResults(v)

_____________
'ich' -> ["['ich']"]
_____________
'du' -> ["['Du']", "['du']"]
_____________
'er' -> ["['er']"]
_____________
'sie' -> ["['sie']"]
_____________
'es' wurde gefunden in: ["['Abendessen']", "['Adresse, Anschrift']", "['Alles']", "['Angestellter']", "['Anästhesie']", "['Atembeschwerden']", "['Bescheid sagen']", "['Besen']", "['Besitz']", "['Besteck']", '...']
_____________
'wir' -> ["['wir']"]
_____________
'ihr' -> ["['ihr', 'Besitz']", "['ihr']"]
_____________
'gehörlos' -> ["['gehörlos']"]
_____________
'wir beide' -> ["['wir beide', 'Person und ich']", "['wir beide', 'du und ich']"]
_____________
'mein' -> ["['mein']"]
_____________
'dein' -> ["['dein']"]
_____________
'sein, ihr' -> ["['sein, ihr']"]
_____________
'unser' -> ["['unser']"]
_____________
'euer' -> ["['euer']"]
_____________
'Name' -> ["['Name']"]
_____________
'was' -> ["['was']"]
_____________
'Körper' -> ["['Körper']"]
_____________
'gut' -> ["['Gut']", "['gut']"]
_____________
'wie geht's' -> ['["wie 

Nun müssen wir manuell jeden Eintrag löschen, der uns nur schlechte Ergebnisse gegeben hat. Einträge, die gar nichts ergeben haben, können wir ignorieren - diese werden am Schluss einfach nicht zum Deck hinzugefügt.

In [43]:
toRemove = [
    'es',
    'nach',
    'mittel',
    'fit',
]

So löscht man alle zum Löschen gesammelten Einträge:

In [44]:
for rem in toRemove:
    vocablist.remove(rem)

So ersetzt man alte Einträge durch neue, um die Schreibweise an die SignDict-Datenbank anzupassen:

In [45]:
def replace(fromList, oldValue, newValue):
    assert oldValue in fromList, "Der angegebene Eintrag muss in der Liste sein. Bitte Rechtschreibung prüfen"
    return [newValue if x==oldValue else x for x in fromList]

In [46]:
vocablist = replace(vocablist, 'Gebärde', 'gebärden')
vocablist = replace(vocablist, 'Schwindel', 'schwindlig')

Und so fügt man einen zusätzlichen Eintrag hinzu, nachdem man geprüft hat, dass es die Vokabel auch auf SignDict gibt:

In [47]:
zusatzVocab = 'frisch'
getResults(zusatzVocab)

_____________
'frisch' -> ["['frisch']"]


In [48]:
vocablist.append(zusatzVocab)

### Wir sind nun zufrieden mit den Deckeinträgen

Daher können wir nun die folgenden Zellen laufen lassen:

In [50]:
def addKeys(keylist, vocab):
    perfectmatches = list(sorted(set([tuple(x[1:]) for x in entrylist if vocab.lower() == x[1].lower()])))
    if len(perfectmatches)>0:
        keylist += perfectmatches
        return
    okaymatches = list(sorted(set([tuple(x[1:]) for x in entrylist if vocab.lower() in x[1].lower()])))
    if len(okaymatches)>0:
        keylist += okaymatches
        return

In [51]:
vocabKeys = []
for v in vocablist:
    addKeys(vocabKeys, v)
vocabKeys[:5] + ['...']

[('ich',), ('Du',), ('du',), ('er',), ('sie',), '...']

Zu guter Letzt generieren wir noch die Anki-Datei!

In [52]:
createDeck('Kapitel 1', '6 ViKo DGS Kapitel 1', vocabKeys)

Und: Daran denken, das Deck noch manuell zu mischen, bevor du es verwendest.

# Das war's! Bei weiteren Fragen bitte auf [GitHub](https://github.com/SamuelVilz/anki-german-sign-language) einen "Issue" öffnen